In [36]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.spatial import ConvexHull
import cv2
from scipy.spatial.distance import cdist

### UTILITY FUNCTIONS

In [49]:
# Reads a particular csv file and returns a list of curves
def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
            path_XYs.append(XYs[0])
    return path_XYs

# Plots a list of curves
def plot(paths_XYs):
    fig, ax = plt.subplots(tight_layout=True, figsize=(8, 8))
    colours = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for i, XYs in enumerate(paths_XYs):
        c = colours[i % len(colours)]
        for XY in XYs:
            ax.plot(XY[:, 0], XY[:, 1], c=c, linewidth=2)
    ax.set_aspect('equal')
    plt.show()

# Reads a csv file and plots the shape detailed there
def plot_shape(csv_path):
    csv_curve = read_csv(csv_path)
    plot(csv_curve)
    print("Number of curves:", len(csv_curve))
    
# Returns the corner points for a set of points
def get_corners(points, epsilon_factor=0.02):
    points = np.array(points, dtype=np.float32).reshape((-1, 1, 2))
    perimeter = cv2.arcLength(points, True)    
    # Approximate the contour with a polygon
    epsilon = epsilon_factor * perimeter
    approx = cv2.approxPolyDP(points, epsilon, True)    
    # Extract corners from the approximation
    corners = [tuple(pt[0]) for pt in approx]
    return corners

def plot_shape_and_corners(points, corners):
    points = np.array(points)
    plt.figure(figsize=(8, 6))
    plt.plot(points[:, 0], points[:, 1], 'o-', label='Original Shape')
    corners = np.array(corners)
    plt.plot(corners[:, 0], corners[:, 1], 'ro', label='Corners')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    plt.title('Shape and Corners')
    plt.show()

def plot_convex_hull(points):
    points = np.array(points)    
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]    
    plt.figure(figsize=(8, 8))
    plt.scatter(points[:, 0], points[:, 1], color='red', s=10, label='Original Points')
    plt.plot(np.append(hull_points[:, 0], hull_points[0, 0]), np.append(hull_points[:, 1], hull_points[0, 1]), color='blue', linestyle='-', label='Convex Hull')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title('Original Points and Convex Hull')
    plt.show()

### Laying out the foundation

We have multiple filters that will act on the doodle one by one to alter the array of points and the point mask (which indicates whether the subsequent filter should act on a given point or not)

In [54]:
class Filter:
    def __init__(self):
        pass
    def __call__(self, points, pointMask):
        return points, pointMask
    
class Mask:
    def __init__(self):
        self.data = dict()
        
    @staticmethod
    def convertKey(key):
        return tuple(list(key))
    
    def __getitem__(self, key):
        return self.data[Mask.convertKey(key)]
    
    def __setitem__(self, key, value):
        self.data[Mask.convertKey(key)] = value
        
    def __delitem__(self, key):
        del self.data[Mask.convertKey(key)]
        
    
class Doodle:
    def __init__(self, curves):
        self.curves = curves
        self.pointMask = Mask()
        for curve in self.curves:
            for point in curve:
                self.pointMask[point] = 1
        self.filters = []
        
    def addFilter(self, filter : Filter):
        self.filters.append(filter)
        return self
    
    def filteredPoints(self):
        curves =  [[point for point in curve if self.pointMask[point] == 1] for curve in self.curves]
        curves = [curve for curve in curves if len(curve) > 0]
        return curves
    
    def build(self):
        for filter in self.filters:
            self.curves, self.pointMask = filter(self.filteredPoints(), self.pointMask)
        return self

In [55]:
doodle = Doodle(read_csv("./problems/problems/frag0.csv"))

### Filters for regular curves

In [56]:
# This filter straightens all straight lines. It doesn't remove them, just straightens them
class LineFilter(Filter):
    def is_straight_line(self, curve, threshold=0.99):
        x = np.array([p[0] for p in curve]).reshape(-1, 1)
        y = np.array([p[1] for p in curve])
        
        model = LinearRegression().fit(x, y)
        y_pred = model.predict(x)
        
        r2 = r2_score(y, y_pred)
        return r2 >= threshold
    
    def fitline(self, curve):
        return curve
    
    def __call__(self, curves, pointMask):
        newcurves = []
        for curve in curves:
            newcurve = curve
            if self.is_straight_line(curve) : 
                newcurve = self.fitline(curve)
            newcurves.append(curve)
            
        return newcurves, pointMask

In [ ]:
class ConvexFilter(Filter):
    def is_straight_line(self, curve, threshold=0.99):
        x = np.array([p[0] for p in curve]).reshape(-1, 1)
        y = np.array([p[1] for p in curve])
        
        model = LinearRegression().fit(x, y)
        y_pred = model.predict(x)
        
        r2 = r2_score(y, y_pred)
        return r2 >= threshold
    
    def fitline(self, curve):
        return curve
    
    def __call__(self, curves, pointMask):
        newcurves = []
        for curve in curves:
            newcurve = curve
            if self.is_straight_line(curve) : 
                newcurve = self.fitline(curve)
            newcurves.append(curve)
            
        return newcurves, pointMask

In [57]:
doodle = doodle.addFilter(LineFilter()).build()